<a href="https://colab.research.google.com/github/mohammad-hosein/Semi-Blind-MLP/blob/main/Copy_of_Semi_blind_MLP_Paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import math
import pandas as pd
import tensorflow as tf
import random
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier

# Generating datasets and preprocessing

In [2]:
'''Fading channel for specific number of antennas at transmitter and receiver side'''
j = complex(0, 1) #Imaginary part
def fading_channel (Nt, Nr):
  h_real = np.random.normal(0, 0.5, size = (Nr, Nt))
  h_img = np.random.normal(0, 0.5, size = (Nr, Nt))
  H_Rayleigh = np.array(h_real + h_img*j, dtype = complex)
  
  return H_Rayleigh

In [3]:
'''SSK Constellation set'''
Constellation_set= np.zeros(shape= (4, 4))
Constellation_set[0] = np.array([1, 0, 0, 0])
Constellation_set[1] = np.array([0, 1, 0, 0])
Constellation_set[2] = np.array([0, 0, 1, 0])
Constellation_set[3] = np.array([0, 0, 0, 1])

In [4]:
H_Rayleigh = fading_channel (4,2)

In [5]:
'''Complex noise'''
def noise_complex (N0, Nr):
  n_real = np.random.normal(0, N0, size = Nr)
  n_img=np.random.normal(0, N0, size = Nr)
  noise_complex=np.array(n_real + n_img*j, dtype = complex)
  
  return noise_complex

In [6]:
'''Creating pilot signals'''
def training_pilots_at_receiver (Label):
  i = 0
  
  y_receiver = np.zeros(shape = (4, 2), dtype = "complex_")
  for i in range(Label):
    y_receiver[i] = np.dot(H_Rayleigh, Constellation_set[i], out = None) + noise_complex (0.125, 2)
    i += 1

  y_receiver_real = y_receiver.real
  y_receiver_imag = y_receiver.imag
  y_receiver = np.concatenate((y_receiver_real, y_receiver_imag), axis = 1)
  y_receiver_complex = pd.DataFrame(y_receiver, columns=['Ant1_real', 'Ant2_real', 'Ant1_imag', 'Ant2_imag'])
  y_receiver_complex['Label'] = np.arange(0, 4) 

  return y_receiver_complex

In [7]:
training_pilots_at_receiver = training_pilots_at_receiver (4)

In [8]:
'''Creating test signals'''
def test_ (packet_length):
  #Define an empty dataframe
  y_receive_test = pd.DataFrame()

  for i in range(packet_length):
    #Generating received signals
    y_one_receive_signal = np.zeros(shape = (1, 2), dtype = "complex_")
    
    #Choosing a random constellation point
    rand = random.randrange(0, 4, 1)
    
    #Y = HX+N 
    y_one_receive_signal = np.dot(H_Rayleigh, Constellation_set[rand], out = None) + noise_complex (0.125, 2)
    
    #Transforming complex signal to a two dimension integer
    y_one_receive_signal_real = y_one_receive_signal.real
    y_one_receive_signal_imag = y_one_receive_signal.imag
    y_receive = np.concatenate((y_one_receive_signal_real,y_one_receive_signal_imag), axis = None)
    
    #Filling dataframe rows
    y_receive_without_label = pd.DataFrame(y_receive.reshape((1, 4)), columns = ['Ant1_real', 'Ant2_real', 'Ant1_imag', 'Ant2_imag'])
    
    #Assigning related label to received pilots
    if rand == 0 :
      y_receive_without_label['Label']= 0
    elif rand == 1 :
      y_receive_without_label['Label']= 1
    elif rand == 2 :
      y_receive_without_label['Label']= 2
    elif rand == 3 :
      y_receive_without_label['Label']= 3
    
    #Join all test signals to data set
    y_receive_test = pd.concat([y_receive_test, y_receive_without_label])

  return y_receive_test

In [9]:
def test_label (KNN_or_NN): #Depend on model
  test = test_ (2000)

  #Test data for KNN
  if KNN_or_NN == 1 :
    test_features = test[['Ant1_real', 'Ant2_real', 'Ant1_imag', 'Ant2_imag']]
    test_label = test['Label']
    return test_features, test_label

  #Test data for NN
  elif KNN_or_NN == 0:
    test_features = test[['Ant1_real', 'Ant2_real', 'Ant1_imag', 'Ant2_imag']]
    test_label = test['Label']
    test_label_categorical = to_categorical(test_label, num_classes = 4)
    return test_features, test_label_categorical

In [10]:
test_label (1)

(    Ant1_real  Ant2_real  Ant1_imag  Ant2_imag
 0   -0.856114  -0.344164   0.907159   0.773764
 0   -0.332707  -0.506928  -0.410155   0.078732
 0   -0.175042  -0.546794  -0.494274   0.025536
 0   -0.743285  -0.479061   0.826285   1.163410
 0   -0.127160   0.171018   0.014731  -0.118574
 ..        ...        ...        ...        ...
 0    0.667080   1.065540  -0.545652   0.065518
 0    0.776965   1.053215  -0.254048  -0.088903
 0    0.120220   0.305397   0.077749  -0.038119
 0    0.020923  -0.642162  -0.502672   0.155027
 0    0.722815   1.136862  -0.246725   0.038804
 
 [2000 rows x 4 columns], 0    3
 0    2
 0    2
 0    3
 0    0
     ..
 0    1
 0    1
 0    0
 0    2
 0    1
 Name: Label, Length: 2000, dtype: int64)

In [11]:
test_label (0)

(    Ant1_real  Ant2_real  Ant1_imag  Ant2_imag
 0   -0.188809  -0.607321  -0.503751   0.119441
 0   -0.070592  -0.643374  -0.333775   0.266426
 0   -0.533375  -0.239119   0.985562   1.123412
 0   -0.610255  -0.311948   1.073597   0.958784
 0   -0.004273   0.429668   0.128100  -0.312671
 ..        ...        ...        ...        ...
 0   -0.871003  -0.376017   0.849274   0.881685
 0   -0.072468   0.216846  -0.043437  -0.327736
 0    0.817279   0.892804  -0.022251  -0.081220
 0   -0.024746   0.257008  -0.403140  -0.147276
 0    0.003561   0.576559  -0.050196  -0.051164
 
 [2000 rows x 4 columns], array([[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        ...,
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.]], dtype=float32))

In [12]:
x_test_svm, y_test_svm = test_label (1)
x_test_nn, y_test_nn = test_label (0)


# Generating virtual pilots (data augmentation)

In [13]:
'''Noise for data augmentation'''
def noise_real (N0,Nr):
  noise_real=np.random.normal(0, N0, size=Nr)
  
  return noise_real

In [14]:
def data_augmentation (sample_size, noise_variance):
  #Generating virtual pilots based on actual pilots
  augmented_data = pd.DataFrame()

  for i in range(sample_size):
    frame_augmented = training_pilots_at_receiver[['Ant1_real', 'Ant2_real', 'Ant1_imag', 'Ant2_imag']] + noise_real(noise_variance, 4)
    frame_augmented['Label'] = np.arange(0, 4)
    augmented_data = augmented_data.append(frame_augmented)
    i += 1

  return augmented_data

In [15]:
data_augmentation = data_augmentation(100, 0.03)

In [16]:
print(data_augmentation.describe())
print(training_pilots_at_receiver.describe())

        Ant1_real   Ant2_real   Ant1_imag   Ant2_imag      Label
count  200.000000  200.000000  200.000000  200.000000  200.00000
mean    -0.098847    0.113772    0.074211    0.057166    1.50000
std      0.397495    0.678374    0.482940    0.558470    1.12084
min     -0.687566   -0.573635   -0.441796   -0.387693    0.00000
25%     -0.357358   -0.474591   -0.294604   -0.277388    0.75000
50%     -0.118179   -0.088410   -0.074925   -0.247907    1.50000
75%      0.141125    0.505384    0.309222    0.103390    2.25000
max      0.531705    1.235156    0.956736    1.091829    3.00000
       Ant1_real  Ant2_real  Ant1_imag  Ant2_imag     Label
count   4.000000   4.000000   4.000000   4.000000  4.000000
mean   -0.097410   0.115766   0.080087   0.061724  1.500000
std     0.456530   0.780589   0.555046   0.642208  1.290994
min    -0.625053  -0.521069  -0.375464  -0.276218  0.000000
25%    -0.325780  -0.449041  -0.265302  -0.269908  0.750000
50%    -0.117534  -0.093819  -0.086110  -0.250764  1.50

# Split dataset to train and test

In [17]:
'''Spliting features and labels'''
#xy=pd.concat([data_augmentation, training_pilots_at_receiver])       #Enable when you want vp for SVM & KNN
xy = training_pilots_at_receiver                      #Enable when you dont want vp for SVM & KNN
xy_NN = pd.concat([data_augmentation, training_pilots_at_receiver])     #Enable when you want vp for NN
#xy_NN = training_pilots_at_receiver                   #Enable when you dont want vp for NN

In [18]:
xy

,Ant1_real,Ant2_real,Ant1_imag,Ant2_imag,Label
0,-0.009046,0.237392,0.056360,-0.267805,0
1,0.470483,1.171771,-0.228581,-0.233724,1
2,-0.226022,-0.521069,-0.375464,-0.276218,2
3,-0.625053,-0.425031,0.868034,1.024641,3


#Preprocessing for KNN & SVM

In [19]:
scaler = StandardScaler()

In [20]:
scaler.fit(xy.drop('Label',axis=1))

StandardScaler()

In [21]:
scaled_features = scaler.transform(xy.drop('Label',axis=1))

In [22]:
xy = pd.DataFrame(scaled_features,columns=xy.columns[:-1])
xy['Label']=np.arange(0,4)
xy

,Ant1_real,Ant2_real,Ant1_imag,Ant2_imag,Label
0,0.223497,0.179918,-0.049361,-0.592497,0
1,1.436370,1.562114,-0.642144,-0.531220,1
2,-0.325298,-0.942049,-0.947717,-0.607625,2
3,-1.334568,-0.799983,1.639222,1.731342,3


#Preprocessing for MLP

In [23]:
xy_NN

,Ant1_real,Ant2_real,Ant1_imag,Ant2_imag,Label
0,0.000803,0.247236,0.024779,-0.252285,0
1,0.480332,1.181614,-0.260162,-0.218204,1
2,-0.216172,-0.511226,-0.407046,-0.260699,2
3,-0.615204,-0.415187,0.836453,1.040160,3
0,-0.004665,0.277073,0.057779,-0.238250,0
...,...,...,...,...,...
3,-0.656707,-0.420142,0.819913,1.045025,3
0,-0.009046,0.237392,0.056360,-0.267805,0
1,0.470483,1.171771,-0.228581,-0.233724,1
2,-0.226022,-0.521069,-0.375464,-0.276218,2


In [24]:
'''preprocessing for NN'''
x_NN=xy_NN[['Ant1_real','Ant2_real','Ant1_imag','Ant2_imag']]
y_NN=xy_NN['Label']
'''Labels one hot encoding'''
y_NN=to_categorical(y_NN, num_classes=4)
'''Spliting data to train and validation'''
X_train_NN, X_test_NN, y_train_NN, y_test_NN = train_test_split(x_NN, y_NN, test_size=0.3, random_state=101)

In [25]:
'''Spliting features and labels for SVM & KNN'''
x=xy[['Ant1_real','Ant2_real','Ant1_imag','Ant2_imag']]
y=xy['Label']

In [26]:
xy

,Ant1_real,Ant2_real,Ant1_imag,Ant2_imag,Label
0,0.223497,0.179918,-0.049361,-0.592497,0
1,1.436370,1.562114,-0.642144,-0.531220,1
2,-0.325298,-0.942049,-0.947717,-0.607625,2
3,-1.334568,-0.799983,1.639222,1.731342,3


In [27]:
'''Spliting data to train and validation'''
#X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.000001, random_state=101)
X_train=x
y_train=y

In [28]:
X_train.shape

(4, 4)

In [29]:
y_train.shape

(4,)

# Building neural network and training

In [30]:
model = Sequential()
model.add(Input(shape=(4)))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
#model.add(Dropout(rate=0.1))
#model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='softmax'))


In [31]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
NN=model.fit(X_train_NN, y_train_NN, batch_size=8, epochs=10, validation_data=(X_test_NN,y_test_NN))

Epoch 1/10
18/18 [==============================] - 2s 22ms/step - loss: 1.3004 - accuracy: 0.4085 - val_loss: 1.1646 - val_accuracy: 0.7903
Epoch 2/10
18/18 [==============================] - 0s 4ms/step - loss: 1.0967 - accuracy: 0.7254 - val_loss: 0.9263 - val_accuracy: 0.8226
Epoch 3/10
18/18 [==============================] - 0s 5ms/step - loss: 0.8831 - accuracy: 0.8099 - val_loss: 0.6760 - val_accuracy: 0.9194
Epoch 4/10
18/18 [==============================] - 0s 5ms/step - loss: 0.6548 - accuracy: 0.9085 - val_loss: 0.4582 - val_accuracy: 0.9677
Epoch 5/10
18/18 [==============================] - 0s 4ms/step - loss: 0.4820 - accuracy: 0.9859 - val_loss: 0.3055 - val_accuracy: 1.0000
Epoch 6/10
18/18 [==============================] - 0s 5ms/step - loss: 0.3563 - accuracy: 0.9789 - val_loss: 0.2049 - val_accuracy: 1.0000
Epoch 7/10
18/18 [==============================] - 0s 12ms/step - loss: 0.2481 - accuracy: 1.0000 - val_loss: 0.1337 - val_accuracy: 1.0000
Epoch 8/10
18/18 [

In [32]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                160       
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 4)                 132       
                                                                 
Total params: 4,484
Trainable params: 4,484
Non-trainable params: 0
_________________________________________________________________


# Training KNN and SVM model

In [33]:
rbf = svm.SVC(kernel = 'rbf', gamma = 0.5, C = 0.1).fit(X_train, y_train)
poly = svm.SVC(kernel = 'poly', degree = 3, C = 1).fit(X_train, y_train)
knn = KNeighborsClassifier(n_neighbors=1).fit(X_train,y_train)

# Testing models on test dataset 

In [34]:
poly_pred = poly.predict(x_test_svm)
rbf_pred = rbf.predict(x_test_svm)
knn_pred = knn.predict(x_test_svm)

In [35]:
knn_accuracy = accuracy_score(y_test_svm, knn_pred)
knn_f1 = f1_score(y_test_svm, knn_pred, average ='weighted')
print('Accuracy (KNN): ', "%.2f" % (knn_accuracy*100))
print('F1 (KNN): ', "%.2f" % (knn_f1*100))

Accuracy (KNN):  85.65
F1 (KNN):  85.21


In [36]:
poly_accuracy = accuracy_score(y_test_svm, poly_pred)
poly_f1 = f1_score(y_test_svm, poly_pred, average = 'weighted')
print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))

Accuracy (Polynomial Kernel):  24.50
F1 (Polynomial Kernel):  9.64


In [37]:
rbf_accuracy = accuracy_score(y_test_svm, rbf_pred)
rbf_f1 = f1_score(y_test_svm, rbf_pred, average = 'weighted')
print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
print('F1 (RBF Kernel): ', "%.2f" % (rbf_f1*100))

Accuracy (RBF Kernel):  85.65
F1 (RBF Kernel):  85.21


In [38]:
'''Test the model'''
NN_Accuracy = model.evaluate(x_test_nn, y_test_nn)

63/63 [==============================] - 0s 5ms/step - loss: 0.0655 - accuracy: 0.9880


#Choosing K value

In [39]:
'''error_rate = []

for i in range(1,4):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    pred_i = knn.predict(m1)
    error_rate.append(np.mean(pred_i != n1))'''

'error_rate = []\n\nfor i in range(1,4):\n    knn = KNeighborsClassifier(n_neighbors=i)\n    knn.fit(X_train,y_train)\n    pred_i = knn.predict(m1)\n    error_rate.append(np.mean(pred_i != n1))'

In [40]:
'''plt.figure(figsize=(10,6))
plt.plot(range(1,40),error_rate,color='blue',ls='--',marker='o',markerfacecolor='red',markersize=10)
plt.title('Error Rate vs. K Values')
plt.xlabel('K')
plt.ylabel('Error Rate')'''

"plt.figure(figsize=(10,6))\nplt.plot(range(1,40),error_rate,color='blue',ls='--',marker='o',markerfacecolor='red',markersize=10)\nplt.title('Error Rate vs. K Values')\nplt.xlabel('K')\nplt.ylabel('Error Rate')"